### Histogram of counted volumes per roi using pypore3d

In [1]:
import os
import random 
from glob import glob 
import numpy as np 
import matplotlib.pyplot as plt 
from tqdm import tqdm 
import json

from scipy.stats import norm

from poreUtils import *


In [2]:
def NormalizeData(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))


In [3]:
# Directory where the scans are stored with results 
root_dir = 'D:\sagar\Data'

In [4]:
# Getting the scans where pypore3d analysis has been run 
scans = []
for p in os.listdir(root_dir):
    file_path = os.path.join(root_dir, p, 'result')
    if os.path.exists(file_path):
        scans.append(p)
                

In [5]:
# Getting all the blob volumes to find the global minimum and maximum for scaling
for s in scans:
    result_path = os.path.join(root_dir, s, 'result')
    json_path = glob(result_path + '\*.json')
    blobs = []
    for f in json_path:
        file = open(f)
        data = json.load(file)
        blobs += data['blob_analysis']['result']['VOLUME']
        file.close()

global_min = min(blobs)
global_max = max(blobs)

print(global_min, global_max)

8e-09 0.09666240000000001


In [6]:
# Normalization function for global min max 
def globNorm(data, gmin, gmax):
    return (data - gmin) / (gmax - gmin)

In [7]:

import ipywidgets as widgets 
from ipywidgets import VBox, HBox

In [8]:
# Menu for selecting the sacns for plotting the histogram  
select_scan = widgets.SelectMultiple(
    options = scans,
    value = [scans[0]],
    description = 'Scan',
    disabled = False,
    rows = 10
)

# Text box for min range for histogram plot
min_range = widgets.BoundedFloatText(
    value=0.000001,
    min=0,
    max=10.0,
    step=0.0000005,
    description='range: min:',
    disabled=False
)

# Text box for max range for histogram plot
max_range = widgets.BoundedFloatText(
    value=0.000005,
    min=0,
    max=1,
    step=0.0000005,
    description='range: max:',
    disabled=False
)

# Slider interface for selecting the bins for the histogram 
slider = widgets.IntSlider(min=1, max=200, value=100, description='Bin')

# Toggle button for normalization on the x axis 
normX_btn = widgets.ToggleButton(
            value=False,
            description='Glo.Norm. X',
            disabled=False,
            )

# Toggle button for normalization on the y axis 
normY_btn = widgets.ToggleButton(
            value=True,
            description='Norm. Y',
            disabled=False,
            )
# Layout of the control options 
ui = VBox([HBox([select_scan, normX_btn, normY_btn]), HBox([slider, min_range, max_range])])

# Function that is used in interactive mode 
def his_cmp(picked_scans, h_bin, rn_min, rn_max, normX, normY, gbmin=global_min, gbmax=global_max):
    
    plt.figure(figsize=(16,9))
    
    for aFile in picked_scans:
        fpath = os.path.join(root_dir, aFile, 'result')
        fpath = glob(fpath+'\*.json')
        blob_volumes = []
        for f in fpath:
            file = open(f)
            data = json.load(file)
            blob_volumes += data['blob_analysis']['result']['VOLUME']
            file.close()
        if normX: 
            blob_volumes = np.asarray(blob_volumes)
            blob_volumes = globNorm(blob_volumes, gbmin, gbmax)

        if normY:
            weights = np.ones_like(blob_volumes)/float(len(blob_volumes))
            #density= True
        else:
            weights = None
            #density = False
        
        plt.hist(blob_volumes, bins=h_bin, range=(rn_min, rn_max), weights=weights, histtype='step', label=aFile)
        plt.legend()

    plt.show()
    
out = widgets.interactive_output(his_cmp, {'picked_scans':select_scan, 'h_bin':slider, 'rn_min':min_range, 'rn_max':max_range, 'normX':normX_btn, 'normY':normY_btn})

display(ui, out)

Output()